In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

**Direcciones

In [3]:
train3_folder = (path/'train'/'3').ls() # dirección de las imágenes
train7_folder = (path/'train'/'7').ls()
valid3_folder = (path/'valid'/'3').ls()
valid7_folder = (path/'valid'/'7').ls()

**Creación de los tensores

In [4]:
def stack(folder):
    x = torch.stack([tensor(Image.open(o)) for o in (folder)]).float()/255 # crea una lista con las imágenes (puedo abrir cualquiera con show_image(train_7[])). Divie por 255 para que los valores estén entre 0 y 1
    return x

In [5]:
train_7 = stack(train7_folder) # tensor de rango-3 (6265 matrices de 28x28, siendo cada matriz una imagen)
train_3 = stack(train3_folder) # train_3.shape -> torch.Size([6265, 28, 28]) Se tienen así dos sets de entrenamiento, uno para el 3 y otro para el 7
valid_3 = stack(valid3_folder) # valid_3.shape -> torch.Size([1010, 28, 28])
valid_7 = stack(valid7_folder) # valid_7.shape -> torch.Size([1028, 28, 28]) Se tienen así dos sets de validación, uno para el 3 y otro para el 7

In [6]:
train_x = torch.cat([train_3, train_7]).view(-1, 28*28) # concatena los tensores tridimensionales de entrenamiento, y los convierte en una lista de vectores. Las matrices de 28x28 ahora son vectores de largo 28x28=784
train_y = tensor([1]*len(train3_folder) + [0]*len(train7_folder)).unsqueeze(1) # crea un vector con tantos 1 como elementos hay en la train3_folder, y tantos 0 como elementos hay en train7_folder

valid_x = torch.cat([valid_3, valid_7]).view(-1, 28*28) # el set de validación se muestra también como una colección de vectores
valid_y = tensor([1]*len(valid3_folder) + [0]*len(valid7_folder)).unsqueeze(1) # básicamente, el vector de labels para valid_x

**Creación de los datasets

In [7]:
dset = list(zip(train_x, train_y)) # genera un arreglo de tuplas (train_x,train_y) --es un arreglo donde cada vector tiene su correspondiente etiqueta--
dset_valid = list(zip(valid_x, valid_y)) # idem para el set de validación

In [8]:
def init_params(size, std=1.0):
    return (torch.randn(size)*std).requires_grad_()

In [9]:
def linear1(x):
    return x@weights + bias

In [10]:
def mnist_loss(preds, target):
    preds = preds.sigmoid()
    return torch.where(target ==1, 1-preds, preds).mean()

In [11]:
def mse(preds, target):
    preds= preds.sigmoid()
    return ((preds-target)**2).mean()

In [12]:
# Inicialización de parámetros

weights = init_params(28*28,1)
bias = init_params(1)

params = weights, bias

lr = 1

In [13]:
dl = DataLoader(dset, batch_size=56) # crea lotes de elementos, para procesar en paralelo
dl_valid = DataLoader(dset_valid, batch_size=56) # idem, con el set de validación

In [14]:
def calc_grad(x, y, model):
    preds = linear1(x)
    loss = mnist_loss(preds, y) # calcula el loss para las predicciones respecto a sus labels
    loss.backward()

In [34]:
def metric(pred, target):
    return (pred.sigmoid().round().unsqueeze(1) == target).float().mean() # consultar bien esta línea

# pred es la predicción hecha con linear1
# sigmoid() limita los valores entre 0 y 1
# round() los redondea a enteros
# unsqueeze(1) le da la misma forma que target, para poder comparar
# la comparación da en valores booleanos
# .float() los convierte en números
# .mean() saca el promedio

Para usar de ejemplo lo anterior, se puede probar:<br>
batch_x = list(dl)[43][0]<br>
batch_y = list(dl)[43][1]<br>
pred = linear1(batch_x)<br>
pred<br>
pred.sigmoid()<br>
pred.sigmoid().round()<br>
(pred.sigmoid().round().unsqueeze(1)<br>
(pred.sigmoid().round().unsqueeze(1) == batch_y)<br>
(pred.sigmoid().round().unsqueeze(1) == batch_y).float()<br>
(pred.sigmoid().round().unsqueeze(1) == batch_y).float().mean()<br>

In [42]:
def train_epoch(model, lr, params):
    for x, y in dl: # 'x' sería un lote de entrenamiento, 'y' sus labels
        calc_grad(x, y, linear1)
        #print('error rate:', metric(linear1(x),y).item())
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [43]:
def validate_epoch(model):
    accurates = [metric(linear1(x), y) for x,y in dl_valid] # cuenta los aciertos para cada lote
    return round(torch.stack(accurates).mean().item(), 4) # hace un vector con los valores, y calcular el promedio. El 4 es la cantidad de decimales

In [48]:
for i in range(20):
    train_epoch(linear1, lr, params)
    validate_epoch(linear1) # Mientras más se acerca a 1, mejor. Dio al revés del error rate
    print(validate_epoch(linear1), end='\n')

0.9566
0.9595
0.9633
0.9667
0.9686
0.9706
0.973
0.9735
0.9754
0.9768
0.9778
0.9788
0.9788
0.9792
0.9802
0.9807
0.9807
0.9812
0.9812
0.9817
